In [77]:
import pandas as pd
from itertools import chain
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient

In [2]:
data_dir = '../synonyms_from_category/data/'

In [34]:
synonyms_dict = {}


def parse_synonym_dict(file_name):
    read_file = open(data_dir + file_name, 'r+')
    
    while True:
        line = read_file.readline()
        if not line:
            break

        info = line.split("\t")
        k = info[1]
        v = info[2].split(",")
        
        synonyms_dict[k] = v

    read_file.close()

In [35]:
parse_synonym_dict('thesaurusDic_20190123.txt')

In [39]:
df_cate = pd.read_csv(data_dir+'category_candidate.csv', names=['cate'], sep='\t')

In [40]:
df_cate = df_cate.drop_duplicates(subset=['cate'])
df_cate['len_cate'] = df_cate.cate.apply(lambda x: len(x))
# df_cate = df_cate.loc[df_cate.len_cate > 1]
df_cate = df_cate.reset_index(drop=True)

In [41]:
df_cate['key_list'] = None

In [43]:
for i, row in df_cate.iterrows():
    key_list = []
    for k, v in synonyms_dict.items():
        if row.cate == k or row.cate in v:
            key_list.append(k)
    if key_list:
        row['key_list'] = key_list


In [50]:
def make_key_list(cate):
    key_list = []
    for k, v in synonyms_dict.items():
        if cate == k or cate in v:
            key_list.append(k)
    if key_list:
        return key_list
    else:
        None

In [51]:
df_cate['key_list'] = df_cate['cate'].apply(lambda cate: make_key_list(cate))

In [55]:
key_list = df_cate.dropna(subset=['key_list'])['key_list'].tolist()

In [57]:
key_list = [key for key in chain(*key_list)]
key_list = list(set(key_list))

In [106]:
cand_synonym_dict = {}
for k in key_list:
    temp_list = [x for x in synonyms_dict[k] if not x.endswith('용') and not x.endswith('형')]
    cand_synonym_dict[k] = " ".join(temp_list)

In [107]:
df_synonyms = pd.DataFrame.from_dict(cand_synonym_dict, orient='index')

In [108]:
df_synonyms = df_synonyms.reset_index()
df_synonyms.columns = ['k' ,'v']

In [109]:
df_synonyms.columns

Index(['k', 'v'], dtype='object')

In [110]:
es = Elasticsearch()
ic = IndicesClient(es)

In [111]:
def get_body(category):
    _body = {
        "tokenizer" : "nori_tokenizer",
        "filter" : ["lowercase"],
        "attributes" : ["leftPOS", "rightPOS"], 
        "explain" : True,
    }
    _body['text'] = category
    return _body

def get_terms(category):
    res = ic.analyze('nori_vanilla', body=get_body(category=category))
    return res['detail']['tokenizer']['tokens']

In [112]:
df_synonyms['terms'] = df_synonyms.k.apply(lambda category: get_terms(category))
df_synonyms['len_terms'] = df_synonyms.terms.apply(lambda x: len(x))

In [113]:
df_synonyms = df_synonyms.sort_values(by=['len_terms', 'k']).reset_index(drop=True)

In [114]:
df_synonyms.columns

Index(['k', 'v', 'terms', 'len_terms'], dtype='object')

In [115]:
df_synonyms[['k', 'v']].to_csv(data_dir+'gogossing.tsv', sep='\t', index=False)